#### This ipynb is used to test some functions that I'm not familiar with in python langugange## This ipynb is used to test some functions that I'm not familiar with in python langugange

In [75]:
import jax.numpy as np
import numpy as onp
import jax
from functools import partial

In [64]:
def GaussSet(Gauss_Num = 2, cuda=False):
    if Gauss_Num == 2:
        Gauss_Weight1D = [1, 1]
        Gauss_Point1D = [-1/np.sqrt(3), 1/np.sqrt(3)]
       
    elif Gauss_Num == 3:
        Gauss_Weight1D = [0.55555556, 0.88888889, 0.55555556]
        Gauss_Point1D = [-0.7745966, 0, 0.7745966]
    return Gauss_Weight1D, Gauss_Point1D

In [65]:
Gauss_Weight1D, Gauss_Point1D = GaussSet()
print(Gauss_Weight1D)
print(Gauss_Point1D)

[1, 1]
[Array(-0.57735026, dtype=float32, weak_type=True), Array(0.57735026, dtype=float32, weak_type=True)]


zip is used to unpack the variables individually

In [66]:
quad_points, quad_weights = [], []
for ipoint, iweight in zip(Gauss_Point1D, Gauss_Weight1D):
    quad_points.append([ipoint])
    quad_weights.append(iweight)
print('quad_points are', quad_points)
print('quad_weights are', quad_weights)

quad_points are [[Array(-0.57735026, dtype=float32, weak_type=True)], [Array(0.57735026, dtype=float32, weak_type=True)]]
quad_weights are [1, 1]


define a function in the code

what is lambda here?

lambda x: x + 1

<function __main__.<lambda>(x)>

In [67]:
def get_shape_val_functions(elem_type):
    """Hard-coded first order shape functions in the parent domain.
    Important: f1-f8 order must match "self.cells" by gmsh file!
    """
    if elem_type == 'D1LN2N':
        f1 = lambda x: 1./2.*(1 - x[0])
        f2 = lambda x: 1./2.*(1 + x[0]) 
        shape_fun = [f1, f2]
        
    return shape_fun

#output the value
get_shape_val_functions('D1LN2N')[0]([0.5, 1])

0.25

jax.grad()

In [68]:
def get_shape_grad_functions(elem_type):
    shape_fns = get_shape_val_functions(elem_type)
    return [jax.grad(f) for f in shape_fns]
grad = get_shape_grad_functions('D1LN2N')[0]([0.2])
print(grad)
type(grad[0])

[Array(-0.5, dtype=float32, weak_type=True)]


jaxlib.xla_extension.ArrayImpl

In [69]:
def get_quad_points(Gauss_Num, dim):
    #Gauss_Num is the number of gauss point used in the integra
    # This function is compatible with FEM and CFEM
    
    Gauss_Weight1D, Gauss_Point1D = GaussSet(Gauss_Num)
    #get the weights and location for different number of gauss points
    quad_points, quad_weights = [], []

    for ipoint, iweight in zip(Gauss_Point1D, Gauss_Weight1D):
        if dim < 2:
            quad_points.append([ipoint])
            quad_weights.append(iweight)
        else:
            for jpoint, jweight in zip(Gauss_Point1D, Gauss_Weight1D):
                if dim < 3:
                    quad_points.append([ipoint, jpoint])
                    quad_weights.append(iweight * jweight)
                else:
                    for kpoint, kweight in zip(Gauss_Point1D, Gauss_Weight1D):
                        quad_points.append([ipoint, jpoint, kpoint])
                        quad_weights.append(iweight * jweight * kweight)
                            
            
    quad_points = np.array(quad_points) # (quad_degree*dim, dim)
    quad_weights = np.array(quad_weights) # (quad_degree,)
    return quad_points, quad_weights
    #quad_points is a list of list of all the quadrature coor
    #quad_weights is a list of all the weights corresponding to the quadrature point

def get_shape_vals(Gauss_Num, dim, elem_type):
    """Pre-compute shape function values

    Returns
    -------
    shape_vals: ndarray
        (8, 8) = (num_quads, num_nodes)  
    """
    shape_val_fns = get_shape_val_functions(elem_type)
    quad_points, quad_weights = get_quad_points(Gauss_Num, dim)
    shape_vals = []
    for quad_point in quad_points:
        physical_shape_vals = []
        for shape_val_fn in shape_val_fns:
            physical_shape_val = shape_val_fn(quad_point) 
            physical_shape_vals.append(physical_shape_val)
 
        shape_vals.append(physical_shape_vals)

    shape_vals = np.array(shape_vals) # (num_quads, num_nodes)
    # assert shape_vals.shape == (global_args['num_quads'], global_args['num_nodes'])
    return shape_vals

get_shape_vals(3, 1, 'D1LN2N')

Array([[0.8872983 , 0.11270171],
       [0.5       , 0.5       ],
       [0.11270171, 0.8872983 ]], dtype=float32)

Hard part: 
* understand how to compute jocobian
* understand how to do gpu computation

np.take

a = [4, 3, 5, 7, 6, 8]
indices = [0, 1, 4]
np.take(a, indices)
array([4, 3, 6])

In [70]:
XY = np.array([0,1])
Elem_nodes =  np.array([0,1])
np.take(XY, Elem_nodes, axis=0)

Array([0, 1], dtype=int32)

In [76]:
@partial(jax.jit, static_argnames=['Gauss_Num', 'dim', 'elem_type']) # must
def get_shape_grads(Gauss_Num, dim, elem_type, XY, Elem_nodes):
    """Pre-compute shape function gradients

    Returns
    -------
    shape_grads_physical: ndarray
        (cell, num_quads, num_nodes, dim)  
    JxW: ndarray #Jocobian time the weight matrix
        (cell, num_quads)
    """ 
    #XY       :  
    #Ele_nodes:  
    shape_grad_fns = get_shape_grad_functions(elem_type)
    quad_points, quad_weights = get_quad_points(Gauss_Num, dim)
    shape_grads = []
    for quad_point in quad_points:
        physical_shape_grads = []
        for shape_grad_fn in shape_grad_fns:
            # See Hughes, Thomas JR. The finite element method: linear static and dynamic finite element analysis. Courier Corporation, 2012.
            # Page 147, Eq. (3.9.3)
            #\frac{\partial N}{\partial ksi}
            physical_shape_grad = shape_grad_fn(quad_point)
            physical_shape_grads.append(physical_shape_grad)
 
        shape_grads.append(physical_shape_grads)

    shape_grads = np.array(shape_grads) # (num_quads, num_nodes, dim)
    print(shape_grads.shape, shape_grads)
    # assert shape_grads.shape == (global_args['num_quads'], global_args['num_nodes'], global_args['dim'])

    physical_coos = np.take(XY, Elem_nodes, axis=0) # (num_cells, num_nodes, dim)
    print('physical coor is', physical_coos)
    # physical_coos: (num_cells, none,      num_nodes, dim, none)
    # shape_grads:   (none,      num_quads, num_nodes, none, dim)
    # (num_cells, num_quads, num_nodes, dim, dim) -> (num_cells, num_quads, dim, dim)
    jacobian_dx_deta = np.sum(physical_coos[:, None, :, :, None] * shape_grads[None, :, :, None, :], axis=2) # dx/deta
    # print(jacobian_dx_deta.shape)
    jacbian_det = np.linalg.det(jacobian_dx_deta) # (num_cells, num_quads)
    # print(jacbian_det)
    jacobian_deta_dx = np.linalg.inv(jacobian_dx_deta) # (num_cells, num_quads, dim, dim) # deta/dx
    # print(jacobian_deta_dx.shape)
    # (num_cell, num_quads, num_nodes, dim) matmul (num_cells, num_quads, dim, dim) -> (num_cell, num_quads, num_nodes, dim)
    shape_grads_physical = shape_grads[None, :, :, :] @ jacobian_deta_dx
    # print(shape_grads_physical.shape, shape_grads_physical)

    # For first order FEM with 8 quad points, those quad weights are all equal to one
    # quad_weights = 1.
    JxW = jacbian_det * quad_weights[None, :]
    return shape_grads_physical, JxW # (num_cells, num_quads, num_nodes, dim), (num_cells, num_quads)

Gauss_Num = 2
dim = 1
elem_type = 'D1LN2N'
XY = np.array([0,1])
Elem_nodes =  np.array([0,1])
shape_grads_physical, JxW = get_shape_grads(Gauss_Num, dim, elem_type, XY, Elem_nodes)

(2, 2, 1) Traced<ShapedArray(float32[2,2,1])>with<DynamicJaxprTrace(level=1/0)>
physical coor is Traced<ShapedArray(int32[2])>with<DynamicJaxprTrace(level=1/0)>


IndexError: Too many indices for array: 3 non-None/Ellipsis indices for dim 1.